In [1]:
# !pip install -r requirements.txt

In [20]:
!pip install tensorboard==1.15.0

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 30.1 MB/s eta 0:00:00a 0:00:01
--- Logging error ---
Traceback (most recent call last):
  File "/DATA/gupta37/.local/lib/python3.7/site-packages/pip/_internal/utils/logging.py", line 177, in emit
    self.console.print(renderable, overflow="ignore", crop=False, style=style)
  File "/DATA/gupta37/.local/lib/python3.7/site-packages/pip/_vendor/rich/console.py", line 1673, in print
    extend(render(renderable, render_options))
  File "/DATA/gupta37/.local/lib/python3.7/site-packages/pip/_vendor/rich/console.py", line 1305, in render
    for render_output in iter_render:
  File "/DATA/gupta37/.local/lib/python3.7/site-packages/pip/_internal/utils/logging.py", line 134, in __rich_console__
    for line in lines:
  File "/DATA/gupta37/.local/lib/python3.7/site-packages/pip/_vendor/rich/segment.py", line 249, in split_lines
    for segment in segments:
  File

In [5]:
!python3 main.py

[nltk_data] Downloading package stopwords to
[nltk_data]     /DATA/gupta37/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[*] 13608 samples loaded.
[*] 2237 samples loaded.
[*] 2237 samples loaded.
Using cache found in /DATA/gupta37/.cache/torch/hub/pytorch_vision_v0.8.0
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model

In [1]:
!pip install sentence-transformers

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.3 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125919 sha256=1a10e3f9c52612422e2cec97b7e04a1b0c5a047f15be3022a74234b6b504d17f
  Stored in directory: /DATA/gupta37/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers
--- Logging error ---
Traceback (most recent call last):
  File "/DATA/gupta37/.local/lib/python3.7/site-packages/pip/_internal/utils/logging.py", line 177, in emit
    self.console.print(renderable, overflow="ignore", crop=False, style=style)
  File "/DATA/gupta37/.local/lib/python3.7/site-packages/pip/_vendor/rich/console.py", line 1673, in print
    extend(render(renderable, render_options))
  File "/DATA/gupta37/.local/lib/python3.7/site-packages/

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1,2,4,5,7" 

In [2]:
"""
@author: Chonghan Chen <chonghac@cs.cmu.edu>
"""
from os import path as osp
import os
import logging
from PIL import Image
from torch.serialization import save
from args import get_args
from trainer import Trainer
from crisismmd_dataset import CrisisMMDataset, CrisisMMDatasetWithSSE
from mm_models import DenseNetBertMMModel, ImageOnlyModel, TextOnlyModel
import os
import numpy as np
import torch
from torch.nn.modules import activation
from torch.utils.data import DataLoader
from torch import nn
from torch import optim
import time
import argparse
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     /DATA/gupta37/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:

def get_args():
    parser = argparse.ArgumentParser()


    # -------------- Important configs --------------- #
    parser.add_argument('--mode', choices=['both', 'image_only', 'text_only'], default='both')
    parser.add_argument('--task', choices=['task1', 'task2', 'task2_merged'], default='task1')
    parser.add_argument('--learning_rate', default=2e-3, type=float)
    parser.add_argument('--batch_size', default=64, type=int)
    parser.add_argument('--save_dir', default='./output', type=str)
    parser.add_argument('--model_name', default='', type=str)
    parser.add_argument('--consistent_only', action='store_true')

    parser.add_argument('--with_sse', action='store_true')

    # only used when with_sse set
    parser.add_argument('--pv', default=1000, type=int)
    parser.add_argument('--pt', default=1000, type=int)
    parser.add_argument('--pv0', default=0.3, type=float)
    parser.add_argument('--pt0', default=0.7, type=float)

    # Loading model 
    parser.add_argument('--model_to_load', default='')
    parser.add_argument('--image_model_to_load', default='')
    parser.add_argument('--text_model_to_load', default='')


    parser.add_argument('--max_iter', default=5, type=int)

    # -------------- Default ones --------------- #
    # Running configs

    # Run flag
    parser.add_argument('--debug', action='store_true')
    parser.add_argument('--eval', action='store_true')
    parser.add_argument('--use_tensorboard', action='store_true')

    # System configs
    parser.add_argument('--device', default='cuda')
    parser.add_argument('--num_workers', default=0, type=int)


    # data processing
    parser.add_argument('--load_size', default=228, type=int)
    parser.add_argument('--crop_size', default=224, type=int)
    parser.add_argument('--max_dataset_size', default=2147483648, type=int)


    
    return parser.parse_known_args()


In [4]:
if __name__ == '__main__':
    opt, unkown = get_args()

    model_to_load = opt.model_to_load
    image_model_to_load = opt.image_model_to_load
    text_model_to_load = opt.text_model_to_load

    device = opt.device
    num_workers = opt.num_workers

    EVAL = opt.eval
    USE_TENSORBOARD = opt.use_tensorboard
    SAVE_DIR = opt.save_dir
    MODEL_NAME = opt.model_name if opt.model_name else str(int(time.time()))

    MODE = opt.mode
    TASK = opt.task
    MAX_ITER = opt.max_iter
    OUTPUT_SIZE = None 
    if TASK == 'task1':
        OUTPUT_SIZE = 2
    elif TASK == 'task2':
        OUTPUT_SIZE = 8
    elif TASK == 'task2_merged':
        OUTPUT_SIZE = 6
    else:
        raise NotImplemented

    # The authors did not report the following values, but they tried
    # pv, pt in [10, 20000], and pv0, pt0 in [0, 1]
    WITH_SSE = opt.with_sse
    pv = opt.pv # How many times more likely do we transit to the same class
    pt = opt.pt 
    pv0 = opt.pv0  # Probability of not doing a transition
    pt0 = opt.pt0

    # General hyper parameters
    learning_rate = opt.learning_rate
    batch_size = opt.batch_size

    # Create folder for saving
    save_dir = osp.join(SAVE_DIR, MODEL_NAME)
    if not osp.exists(SAVE_DIR):
        os.mkdir(SAVE_DIR)
    if not osp.exists(save_dir):
        os.mkdir(save_dir)


    # set logger
    logging.basicConfig(filename=osp.join(save_dir, 'output_{}.log'.format(int(time.time()))), level=logging.INFO)


    train_loader, dev_loader = None, None
    if not EVAL:
        if WITH_SSE:
            train_set = CrisisMMDatasetWithSSE()
            train_set.initialize(opt, pv, pt, pv0, pt0, phase='train', cat='all',
                                 task=TASK)
        else:
            train_set = CrisisMMDataset()
            train_set.initialize(opt, phase='train', cat='all',
                                 task=TASK)
        train_loader = DataLoader(
            train_set, batch_size=batch_size, shuffle=True, num_workers=num_workers)

    dev_set = CrisisMMDataset()
    dev_set.initialize(opt, phase='dev', cat='all',
                       task=TASK)

    dev_loader = DataLoader(
        dev_set, batch_size=batch_size, shuffle=False, num_workers=num_workers)

    test_set = CrisisMMDataset()
    test_set.initialize(opt, phase='test', cat='all',
                        task=TASK)

    test_loader = DataLoader(
        test_set, batch_size=batch_size, shuffle=False, num_workers=num_workers
    )

    loss_fn = nn.CrossEntropyLoss()
    if MODE == 'text_only':
        model = TextOnlyModel(num_class=OUTPUT_SIZE, save_dir=save_dir).to(device)
    elif MODE == 'image_only':
        model = ImageOnlyModel(num_class=OUTPUT_SIZE, save_dir=save_dir).to(device)
    elif MODE == 'both':
        model = DenseNetBertMMModel(num_class=OUTPUT_SIZE, save_dir=save_dir).to(device)
    else:
        raise NotImplemented

    model = nn.DataParallel(model)
    # The authors did not mention configurations of SGD. We assume they did not use momentum or weight decay.
    optimizer = optim.SGD(model.parameters(), lr=learning_rate)

    # The authors used factor=0.1, but did not mention other configs.
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, factor=0.1, patience=5, cooldown=0, verbose=True)

    trainer = Trainer(train_loader, dev_loader, test_loader,
                      model, loss_fn, optimizer, scheduler, eval=EVAL, device=device, tensorboard=USE_TENSORBOARD, mode=MODE)

    if model_to_load:
        model.load(model_to_load)
        logging.info("\n***********************")
        logging.info("Model Loaded!")
        logging.info("***********************\n")
    if text_model_to_load:
        model.load(text_model_to_load)
    if image_model_to_load:
        model.load(image_model_to_load)

    if not EVAL:
        logging.info("\n================Training Summary=================")
        logging.info("Training Summary: ")
        logging.info("Learning rate {}".format(learning_rate))
        logging.info("Batch size {}".format(batch_size))
        logging.info(trainer.model)
        logging.info("\n=================================================")

        trainer.train(MAX_ITER)

    else:
        logging.info("\n================Evaluating Model=================")
        logging.info(trainer.model)

        trainer.validate()
        trainer.predict()


[*] 13608 samples loaded.
[*] 2237 samples loaded.
[*] 2237 samples loaded.


Using cache found in /DATA/gupta37/.cache/torch/hub/pytorch_vision_v0.8.0
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
 27%|██▋       | 58/213 [02:20<05:18,  2.05s/it] /DATA/gupta37/.local/lib/python3

In [1]:
import torch
import torch.nn.functional as F

class ImagePatcher(torch.nn.Module):
    def __init__(self, patch_size, stride):
        super(ImagePatcher, self).__init__()
        self.patch_size = patch_size
        self.stride = stride
        
    def forward(self, image):
        # Unfold the image into patches
        patches = F.unfold(image, self.patch_size, stride=self.stride)
        
        # Reshape patches to have shape (batch_size, num_patches, patch_dim)
        batch_size, patch_dim, num_patches = patches.shape
        patches = patches.permute(0, 2, 1).contiguous().view(batch_size, num_patches, patch_dim)
        
        return patches

# Example usage
batch_size = 32
image_channels = 3
image_height = 624
image_width = 624
patch_size = 64
stride = 32

# Create a random image
input_image = torch.randn(batch_size, image_channels, image_height, image_width)

# Initialize the ImagePatcher
patcher = ImagePatcher(patch_size, stride)

# Extract patches from the image
patches = patcher(input_image)

print("Patched image shape:", patches.shape)

Patched image shape: torch.Size([32, 324, 12288])


In [1]:
import torch

tensor_to_repeat = torch.rand(40, 768)  # Example tensor with size [40, 768]
num_repeats = 30

# Repeat each row of the tensor along the first dimension and stack them next to the original rows
repeated_tensor = tensor_to_repeat.unsqueeze(1).repeat(1, num_repeats, 1).view(-1, 768)

print(repeated_tensor.size())  # Output: torch.Size([1200, 768])


torch.Size([1200, 768])


In [24]:
import torch

# Example tensor y with size [40]
y = torch.arange(40)  # Replace with your actual tensor

num_repeats = 36

# Repeat each element of the tensor along the first dimension and stack them in the desired pattern
repeated_y = y.unsqueeze(1).repeat(1, num_repeats).view(-1)

print(repeated_y.size())  # Output: torch.Size([1440])

torch.Size([1440])


In [25]:
repeated_y

tensor([ 0,  0,  0,  ..., 39, 39, 39])

In [11]:
repeated_y

tensor([0.8403, 0.4962, 0.4988,  ..., 0.1059, 0.8652, 0.6119])

In [2]:
repeated_tensor

tensor([[0.1772, 0.0974, 0.3920,  ..., 0.9931, 0.2331, 0.5567],
        [0.1772, 0.0974, 0.3920,  ..., 0.9931, 0.2331, 0.5567],
        [0.1772, 0.0974, 0.3920,  ..., 0.9931, 0.2331, 0.5567],
        ...,
        [0.3354, 0.0778, 0.2314,  ..., 0.4546, 0.2189, 0.2337],
        [0.3354, 0.0778, 0.2314,  ..., 0.4546, 0.2189, 0.2337],
        [0.3354, 0.0778, 0.2314,  ..., 0.4546, 0.2189, 0.2337]])

In [7]:
repeated_tensor[40]

tensor([5.7104e-01, 6.9419e-01, 4.9610e-01, 3.6986e-02, 4.3606e-01, 1.8935e-01,
        6.2840e-02, 4.5809e-01, 7.9554e-01, 5.0419e-01, 6.8246e-01, 2.1839e-01,
        5.2679e-01, 7.6919e-01, 1.0877e-01, 7.9852e-01, 9.5645e-02, 2.4431e-01,
        6.6142e-01, 1.9574e-01, 3.6641e-01, 1.0533e-01, 9.7603e-01, 1.8963e-01,
        7.8898e-01, 9.7901e-01, 8.6092e-01, 5.9248e-01, 8.7987e-01, 1.7068e-01,
        2.4865e-01, 5.7405e-01, 3.6602e-01, 5.2501e-02, 8.1018e-01, 6.3171e-01,
        9.4364e-01, 4.0154e-02, 5.3145e-01, 7.9944e-02, 7.9464e-01, 9.7079e-01,
        1.1360e-01, 7.2285e-01, 5.9192e-02, 8.7586e-01, 3.7283e-01, 6.8294e-01,
        6.8903e-01, 6.1682e-01, 4.2660e-01, 5.5722e-02, 3.3103e-01, 3.8915e-01,
        2.1087e-01, 6.5777e-01, 3.7294e-03, 7.3661e-01, 9.7451e-01, 9.3866e-01,
        3.4468e-01, 1.3064e-01, 2.8299e-01, 2.4630e-01, 7.1969e-01, 6.1637e-01,
        9.7601e-01, 4.8285e-01, 3.0111e-02, 1.6904e-01, 7.2407e-02, 8.8599e-01,
        5.0038e-01, 3.8339e-01, 9.3117e-